In [1]:
from google.colab import drive
import glob
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Functions definition (CenterOfMass, Border, Crop, Add)
def CenterOfMass(img):
    height, width = img.shape
    X, Y, WoR, WoC = [], [], [], []
    for i in range(height):
        Nx = [j for j in range(width) if img[i, j] == 255]
        if Nx:
            X.append(sum(Nx))
            WoR.append(len(Nx))
    CoMX = sum(X) // sum(WoR) if WoR else 0

    for j in range(width):
        Ny = [i for i in range(height) if img[i, j] == 255]
        if Ny:
            Y.append(sum(Ny))
            WoC.append(len(Ny))
    CoMY = sum(Y) // sum(WoC) if WoC else 0

    return CoMX, CoMY

def Border(img):
    height, width = img.shape
    indexl = [j for i in range(height) for j in range(width) if img[i, j] != 0]
    indexr = [j for i in range(height) for j in reversed(range(width)) if img[i, j] != 0]
    indext = [i for j in range(width) for i in range(height) if img[i, j] != 0]
    indexb = [i for j in range(width) for i in reversed(range(height)) if img[i, j] != 0]
    x, X, y, Y = min(indexl), max(indexr), min(indext), max(indexb)
    return x, X, y, Y

def Crop(img, x, X, y, Y):
    return img[y:Y, x:X]

def Add(old, new, center, black, i):
    height, width = old.shape
    x, y = (width // 2 - center[0], height // 2 - center[1])
    new_height, new_width = new.shape
    X, Y = (x + new_width, y + new_height)
    a = 1.0 / (i + 1)
    new = cv2.addWeighted(new, a, old[y:Y, x:X], 1 - a, 0)
    black = cv2.addWeighted(black, a, old, 1 - a, 0)
    old[y:Y, x:X] = new
    return old


# File processing
def generator(file_path, file_out, num):
    print(f"Generator is processing file {num}: {file_path} -> Output directory: {file_out}")
    paths = glob.glob(file_out+'/*.png')
    if paths:
        im = cv2.imread(paths[0])
        if im is None:
          print("Failed to read the image:", paths[0])
          return
        y, x, _ = im.shape
        black = np.zeros((y, x), np.uint8)
        GEI = black.copy()
        i = 0
        for path in paths:
            image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if image is not None and not np.all(image == 0):
                x, X, y, Y = Border(image)
                cropped = Crop(image, x, X, y, Y)
                center = CenterOfMass(cropped)
                GEI = Add(GEI, cropped, center, black, i)
                i += 1
        cv2.imwrite(file_out + '/GEI.png', GEI)

# Main execution path
def iterate_directory(directory):
    num = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.MOV'):
                file_out = os.path.join(root, os.path.splitext(file)[0])
                if not os.path.exists(file_out):
                    os.makedirs(file_out)
                file_path = os.path.join(root, file)
                num += 1
                print(f"Processing file: {file_path}")
                print(f"Output directory: {file_out}")
                generator(file_path, file_out, num)

# Specify the directory path in Google Drive
directory = '/content/drive/Shareddrives/TUG dataset/TUG dataset/17/PD/silhouette/PD_side_8'
iterate_directory(directory)

Processing file: /content/drive/Shareddrives/TUG dataset/TUG dataset/17/PD/silhouette/PD_side_8/PD_side_8.MOV
Output directory: /content/drive/Shareddrives/TUG dataset/TUG dataset/17/PD/silhouette/PD_side_8/PD_side_8
Generator is processing file 1: /content/drive/Shareddrives/TUG dataset/TUG dataset/17/PD/silhouette/PD_side_8/PD_side_8.MOV -> Output directory: /content/drive/Shareddrives/TUG dataset/TUG dataset/17/PD/silhouette/PD_side_8/PD_side_8
